# C# 10 Features Summary
Source: [What's new in C# 10](https://docs.microsoft.com/en-us/dotnet/csharp/whats-new/csharp-10)

Nothing big, mostly improvements and ways to redue clutter from files.

Preview features are skipped.

## Reducing the amount of usings in .cs files

[More info](https://endjin.com/blog/2021/09/dotnet-csharp-10-implicit-global-using-directives)

### Global using directives
Save space in all files by putting common usings in a single file with the `global` keyword.

In [ ]:
global using System;
global using System.Collections.Generic;

Or you can add the usings in the **.csproj** file with:
```
<ItemGroup>
  <Using Include="System.IO.Pipes" />
</ItemGroup>
```

### Implicit usings
Set the porperty in the **.csproj** and the compiler will automatically imply the most relevant usings for your type of project.

```
<PropertyGroup>
    <ImplicitUsings>enable</ImplicitUsings>
</PropertyGroup>
```

If you need to remove an implicit using because of some time being ambiguous:
```
<ItemGroup>
  <Using Remove="System.IO.Pipes" />
</ItemGroup>
```

## File-scoped namespace declaration
One less nesting in your files by having the name spae declared without the brackets, just:

`namespace MyNameSpace;`

[Microsoft Docs](https://docs.microsoft.com/en-us/dotnet/csharp/language-reference/keywords/namespace)

## Extended property patterns
Assign values to nested properties within a property pattern.

[More info](https://www.thomasclaudiushuber.com/2021/10/21/c-10-extended-property-patterns/)

In [ ]:
public class Address
{
    public string Street {get; set;}
}
public class Person
{
    public string Name {get; set;}
    public Address Address {get;set;}
}

var person = new Person { Name = "John", Address = new Address{ Street = "5th Avenue"}};

//                     vvvvvvvvvvvvvv Instead of { Address { Street = "Central Park" }}
if (person is Person { Address.Street: "Central Park"})
{
    Console.WriteLine($"Yes, the person lives in '{person.Address.Street}'");
}
else
{
    Console.WriteLine($"No, the person lives in '{person.Address.Street}'");
}

No, the person lives in '5th Avenue'


### Lambda expressions improvements

### Natural type
Allows the code to be more similar to functional programming, where is normal to assign a method to a variable.

[Microsoft Docs](https://docs.microsoft.com/en-us/dotnet/csharp/language-reference/operators/lambda-expressions#natural-type-of-a-lambda-expression)

In [ ]:
// C# 9:
// Func<string, int> parse = (string s) => int.Parse(s);
// C# 10:
var parse = (string s) => int.Parse(s);
Console.WriteLine($"Parse variable is of type {parse.GetType().Name}");

Parse variable is of type Func`2


### Explicit return type
You can now write the return type of a lambda

[Microsoft Docs](https://docs.microsoft.com/en-us/dotnet/csharp/language-reference/operators/lambda-expressions#explicit-return-type)

In [ ]:
// C# 9:
// var choose = (bool b) => b ? 1 : "two"; // ERROR: Can't infer return type
// CH 10:    vvvvvv
var choose = object (bool b) => b ? 1 : "two";

### Attributes
Now you can add attributes to the lambda and to its parameters.

[Microsoft Docs](https://docs.microsoft.com/en-us/dotnet/csharp/language-reference/operators/lambda-expressions#attributes)

In [ ]:
using System.Diagnostics;
using System.Diagnostics.CodeAnalysis;

var choose = [DebuggerStepThrough] object ([NotNull]Person p) => string.IsNullOrWhiteSpace(p.Name) ? 1 : "two";

## Records improvements

### Record structs
Records are reference types. With `struct` you can make them value types

[Microsoft Docs](https://docs.microsoft.com/en-us/dotnet/csharp/language-reference/builtin-types/record)

In [ ]:
public record struct Point2D
{
    public double X {  get; init; }
    public double Y {  get; init; }
}

### Seal ToString()
So that children can't change it.

[Microsoft Docs]()

In [ ]:
public record Patient
{
    public string Name {get;init;}

    public override sealed string ToString(){
        return $"Patient {Name}";
    }
}

public record CardiologyPatient : Patient
{
    public int HeartRate {get;init;}

    public override string ToString()
    {
        return $"Cardiology Patient: {Name}, {HeartRate}";
    }
}

Error: (14,28): error CS0239: 'CardiologyPatient.ToString()': cannot override inherited member 'Patient.ToString()' because it is sealed

## Interpolated strings

### Interpolated string handlers
Write your own interpolated string handler

[Microsoft Docs](https://docs.microsoft.com/en-us/dotnet/csharp/whats-new/tutorials/interpolated-string-handler)

### Interpolated const strings
Now `const strings` may be initialized using string interpolation

[Microsoft Docs](https://docs.microsoft.com/en-us/dotnet/csharp/language-reference/tokens/interpolated)

In [ ]:
const string Version = "1.1.0";
const string TextualVersion = $"The version is {Version}";

## CallerArgumentExpression attribute diagnostics
The new `CallerArgumentExpression` joins the family of [Caller*] attributes. It puts the actual expression being passed to an argument into a `string`.

[Microsoft Docs](https://docs.microsoft.com/en-us/dotnet/csharp/language-reference/attributes/caller-information#argument-expressions)

In [ ]:
using System.Runtime.CompilerServices;

void WhatIsPassed(bool theParameter, [CallerArgumentExpression("theParameter")] string expression = null)
{
    Console.WriteLine($"The caller wrote: {expression}");
}

WhatIsPassed(true);
WhatIsPassed(1 > 0);

bool AMethodThatReturnsABool(int a, int b) => false;
WhatIsPassed(AMethodThatReturnsABool(2, 5));

The caller wrote: true
The caller wrote: 1 > 0
The caller wrote: AMethodThatReturnsABool(2, 5)


## ArgumentNullException.ThrowIfNull helper
[Microsoft Docs](https://docs.microsoft.com/en-us/dotnet/api/system.argumentnullexception.throwifnull?view=net-6.0)

In [ ]:
void DoSomething(string id)
{
    ArgumentNullException.ThrowIfNull(id);
}

DoSomething(null);

Error: System.ArgumentNullException: Value cannot be null. (Parameter 'id')
   at System.ArgumentNullException.Throw(String paramName)
   at System.ArgumentNullException.ThrowIfNull(Object argument, String paramName)
   at Submission#74.DoSomething(String id)
   at Submission#74.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

## Null check with double exclamation mark
Yoiu can check null by adding `!!` at the end of a parameter. That is it.


```
void DoMore(string id !!)
{
    // ...
}

DoMore(null); // Throws ArgumentNullException.
```

This syntax is not supported by Notebooks yet.

## Improvements for structs

### Parameterless constructors and field initializers
You can declare a parameterless constructor in a `struct`.

[Microsoft Docs](https://docs.microsoft.com/en-us/dotnet/csharp/whats-new/csharp-10#improvements-of-structure-types)

[Microsoft Docs | Language Refernece | Struct](https://docs.microsoft.com/en-us/dotnet/csharp/language-reference/builtin-types/struct#parameterless-constructors-and-field-initializers)


In [ ]:
public struct Measurement
{
    public double Value { get; init; }

    public Measurement()
    {
        Value = double.NaN;
    }

    public Measurement(double value)
    {
        Value = value;
    }
 }

 var m1 = new Measurement();
 Console.WriteLine($"M1 value is: {m1.Value}");

 var m2 = new Measurement(11);
 Console.WriteLine($"M2 value is: {m2.Value}");

M1 value is: NaN
M2 value is: 11


### With keyword can now be used with structs
[Microsoft Docs](https://docs.microsoft.com/en-us/dotnet/csharp/language-reference/operators/with-expression)

In [ ]:
//          vvvv This was only possible with records in C# 9
var m3 = m2 with {Value = 22};

Console.WriteLine($"M3 value is: {m3.Value}");

M3 value is: 22


## Assignment and declaration in same deconstruction
You can now declare variables or use existing ones when deconstructing an instance.

[Microsoft Docs](https://docs.microsoft.com/en-us/dotnet/csharp/whats-new/csharp-10#assignment-and-declaration-in-same-deconstruction)

In [ ]:
record Animal(string Name, string AnimalType);
var dog = new Animal("Dog", "Mammal");

// This was not possible before C#10
string name = string.Empty;
(name, string animalType) = dog;

Console.WriteLine($"{name} is a {animalType}");

Dog is a Mammal


## Improved definite assignment
Definite assignment is a rule simply stating that every variable must have a value before it's read from. The process of assigning a value to a variable for the first time is known as initialization. Once the initialization process has taken place, a variable is considered initialized.[Source](https://www.informit.com/articles/article.aspx?p=23211&seqNum=8#)

Now the compiler will not show warning in certain situations.

[Microsoft Docs](https://docs.microsoft.com/en-us/dotnet/csharp/whats-new/csharp-10#improved-definite-assignment)

## Allow AsyncMethodBuilder attribute on methods
Just [read this in Microsft Docs](https://docs.microsoft.com/en-us/dotnet/csharp/whats-new/csharp-10#allow-asyncmethodbuilder-attribute-on-methods). For performance.

## Enhanced #line pragma
A new format for the `#line pragma`. You likely won't use it. Domain-specific languages like Razor use these enhancements to improve the debugging experience.

[Microsoft Docs](https://docs.microsoft.com/en-us/dotnet/csharp/whats-new/csharp-10#enhanced-line-pragma)